### Dead Card Processing Notebook

In [1]:
import pandas as pd
import numpy as np
from BK_Tree import BKTree
import re
import string
from dead_processing_utils import *

In [2]:
input_path = f"data/DeadRecords_PSE_results_final.csv"
input_df = pd.read_csv(input_path)

determination_key = pd.read_csv("data/NoAuthorTest.csv")
determination_key['no_author'] = determination_key['no_author'].apply(lambda x: punctStrip(x))

FileNotFoundError: [Errno 2] No such file or directory: 'data/DeadRecords_PSE_results_final.csv'

In [ ]:
input_df = TaxonNoAuthor(input_df, 'taxon')
input_df['***no_author'] = input_df['***no_author'].apply(lambda x: punctStrip(x))
input_df['is_cultivar'] = input_df['***no_author'].apply(lambda x: detectCultivar(x))
input_df['***no_author'] = input_df.apply(lambda x: x['***no_author'].split()[0] if x['is_cultivar'] else x['***no_author'], axis=1)

### Detect cultivars

In [ ]:
input_df['cultivar'] = input_df['***no_author'].apply(lambda x: detectCultivar(x))

### Determine which taxon records are already present in the DB

In [ ]:
input_df['taxon_missing'] = input_df['***no_author'].apply(lambda x: True if x not in determination_key['no_author'].values else False)
taxon_to_add = input_df[input_df['taxon_missing']]['***no_author'].unique()
taxon_to_add = input_df[input_df['***no_author'].isin(taxon_to_add)]
already_in_db = input_df[~input_df['taxon_missing']]

print("Taxon not in database verbatim: {}".format(len(taxon_to_add))) 

Taxon not in database verbatim: 412


### Use the external BK_Tree rust stuff to find the closest string to a certain precision (account for minor spelling or formatting errors)

Load the tree class

In [ ]:
taxaTree = BKTree()
for name in determination_key['no_author']:
    taxaTree.insert(name)

In [ ]:
taxon_to_add['recommendation'] = taxon_to_add['***no_author'].apply(lambda x: sortOutput(taxaTree.search(x, 2)))

C:\Users\arman\AppData\Local\Temp\ipykernel_5928\1825728159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxon_to_add['recommendation'] = taxon_to_add['***no_author'].apply(lambda x: sortOutput(taxaTree.search(x, 2)))


In [ ]:
subtractions = taxon_to_add[taxon_to_add['recommendation'].apply(lambda x: len(x) > 0)]
taxon_to_add = taxon_to_add[taxon_to_add['recommendation'].apply(lambda x: len(x) == 0)]

In [ ]:
subtractions['***no_author'] = subtractions['recommendation'].apply(lambda x: x[0][0])
subtractions = subtractions.drop('recommendation', axis=1)

already_in_db = pd.concat([subtractions, already_in_db], axis=0, ignore_index=True)

print("taxon already in database w/ spellcheck: {}".format(len(already_in_db)))
print("taxon to add: {}".format(len(taxon_to_add)))

taxon already in database w/ spellcheck: 740
taxon to add: 340


### Add location and collector guess from DB 

In [ ]:
MatchLocation(already_in_db, 'fieldLocCountry')
MatchLocation(taxon_to_add, 'fieldLocCountry')

In [ ]:
collector_key = pd.read_csv("data/Collectors.csv")

In [ ]:
colTree = BKTree()
for name in collector_key['0']:
    colTree.insert(name)

In [ ]:
def is_empty(val):
    return pd.isna(val) or str(val).strip() == ''

already_in_db['***collector_guess'] = already_in_db['fieldCollector'].apply(lambda x: sortOutput(colTree.search(reformatCol(str(x)), 3)) if not is_empty(x) else None)
taxon_to_add['***collector_guess'] = taxon_to_add['fieldCollector'].apply(lambda x: sortOutput(colTree.search(reformatCol(str(x)), 3)) if not is_empty(x) else None)

already_in_db['***collector_guess'] = already_in_db['***collector_guess'].apply(lambda x: x[0][0] if isinstance(x, list) and x else None)
taxon_to_add['***collector_guess'] = taxon_to_add['***collector_guess'].apply(lambda x: x[0][0] if isinstance(x, list) and x else None)

In [ ]:
print(already_in_db['***collector_guess'][3])

None


In [ ]:
already_in_db.to_csv("output/already_in_db.csv", index=False)
taxon_to_add.to_csv("output/taxon_to_add.csv", index=False)